In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

For this example, we only need one piece of training data. We can also write our own poem or play and pass it to the 
network for training if we like. However, here we will be using Shakespeare's play 'Romeo and Juliet'

In [2]:
#keras has this feature to save it as txt
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
#Read, then decode
text = open(path_to_file,'rb').read().decode(encoding='utf-8')
#length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
print(text[:250]) #first 250 characters in the play

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Encoding
- We encode each unique character as a different integer

In [5]:
vocab = sorted(set(text))

In [17]:
print(type(vocab))
print(vocab)
print(len(vocab))

<class 'list'>
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [7]:
#Create a mapping from unique characters to indices
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
#function to convert numeric values to text
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


Creating Training Examples
- Our task is to feed the model a sequence and have it return to us the next character
- THis means, we need to split our text data from above into many shorter sequences that we can pass to the model as training example
- We will use a sequence as input and another sequence as output, where this o/p sequence is the original sequence shifted one letter to the right
- e.g., i/p: Hell and o/p: ello

First step is to create a stream of characters from our text data

In [10]:
seq_length = 100 # length of sequence for training example
examples_per_epoch = len(text)//(seq_length + 1) #for every training example, we use (seq_length + 1) characters as i/p and o/p combined

#Creating training examples/targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Now, we use the batch method to turn this stream of characters into batches of desired length

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder = True) 
#drop_reamainder will drop the remaining characters at the end, that can't be included in the batch of size 101 here

Use these sequences of length 101 and split them into input and output

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1] #hell
    target_text = chunk[1:] #ello
    return input_text, target_text #hell, ello

dataset = sequences.map(split_input_target) # we use map to apply the above function to every entry

In [13]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [14]:
for x,y in dataset.take(2):
    print("\n\nEXAMPLE")
    print('INPUT: ',int_to_text(x))
    print('\nOUTPUT: ',int_to_text(y))



EXAMPLE
INPUT:  First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT:  irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE
INPUT:  are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT:  re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


FInally, we need to make training batches

In [15]:
BATCH_SIZE = 64 #each epoch split into these many different batches
VOCAB_SIZE = len(vocab) #number of unique characters
EMBEDDING_DIM = 256 #Embedding dimension is how big we want every single vector to represent characters/words????
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences
# so it doesn't attempt to shuffle the entire sequence in memory
# it maintains a buffer in which it shuffle elements)
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

Building the Model
- We will use embedding, LSTM and a dense layer 
- Dense layer contains a node for each unique character in our training data. 
- The dense layer will give us a prob. distr. over all nodes

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
        #while making predictions, we donno how long each sequence is gonna be. So we give shape 'None', i.e., it is the length of sequence
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful = True, recurrent_initializer = 'glorot_uniform'),
        # return_sequences = True, will give us output at every single time step, if set to False, only final output will be given
        # glorot_uniform is a good default to pick for the values to start at in the LSTM
        tf.keras.layers.Dense(vocab_size)
    ])
    
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


Creating A Loss Function
- Our Model will o/p (64,sequence_length,65) shaped tensor that represents the prob. distr. of each character, at each 
timestep, for every sequence in the batch

In [18]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch) #ask our model for a prediction on our first batch of training data
    print(example_batch_predictions.shape, '#(batch_size, sequence_length, vocab_length)')

(64, 100, 65) #(batch_size, sequence_length, vocab_length)
